# Chapter 9 Solvers, templates, and implied volatilities

In [47]:
import numpy as np
import scipy.stats as si
from math import exp,log,sqrt,pi

In [94]:
class BSCall:
    def __init__(self,r_,d_,T_,Spot_,Strike_):
        self.__r = r_
        self.__d = d_
        self.__T = T_
        self.__Spot = Spot_
        self.__Strike = Strike_
        
    def __call__(self,Vol):
        d1 = (log(self.__Spot / self.__Strike) + (self.__r + 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
        d2 = (log(self.__Spot / self.__Strike) + (self.__r - 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
    
        return (self.__Spot * exp(-self.__d * self.__T) * si.norm.cdf(d1, 0.0, 1.0) - self.__Strike * 
                exp(-self.__r * self.__T) * si.norm.cdf(d2, 0.0, 1.0))

In [145]:
theCall = BSCall(0.05, 0, 0.25, 50, 50)
theCall(0.41)

4.37472312788827

In [34]:
def Bisection(Target,Low,High,Tolerance,TheFunction):
    x = 0.5*(Low+High)
    y = TheFunction(x)
    
    while abs(y-Target) > Tolerance:
        if y < Target:
            Low = x
            
        elif y > Target:
            High = x
            
        x = 0.5*(Low + High)
        
        y = TheFunction(x)
        
        
    return x

In [149]:
theCall = BSCall(0.05, 0, 0.25, 50, 50)
Bisection(4.37472312788827,0.01,0.6,0.0000000001,theCall)

0.4100000000066939

In [151]:
theCall(Bisection(4.37472312788827,0.01,0.6,0.0000000001,theCall))

4.3747231279541445

In [117]:
class BSCallTWO:
    def __init__(self,r_,d_,T_,Spot_,Strike_):
        self.__r = r_
        self.__d = d_
        self.__T = T_
        self.__Spot = Spot_
        self.__Strike = Strike_
        
    def Price(self,Vol):
        d1 = (log(self.__Spot / self.__Strike) + (self.__r + 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
        d2 = (log(self.__Spot / self.__Strike) + (self.__r - 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
    
        return (self.__Spot * exp(-self.__d * self.__T) * si.norm.cdf(d1, 0.0, 1.0) - self.__Strike * 
                exp(-self.__r * self.__T) * si.norm.cdf(d2, 0.0, 1.0))
    
    def Vega(self,Vol):
        d1 = (log(self.__Spot / self.__Strike) + (self.__r + 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
        return self.__Spot*exp(-0.5*d1*d1)*sqrt(self.__T)/sqrt(2*pi)
        

In [142]:
def NewtonRaphson(Target, Start, Tolerance, Value, Derivative):
    y = Value(Start)
    x = Start
    
    while abs(y - Target) > Tolerance:
        d = Derivative(x)
        x += (Target - y)/d
        y = Value(x)  
        
        
    return x

In [152]:
theCall = BSCallTWO(0.05, 0, 0.25, 50, 50)
NewtonRaphson(4.37472312788827,0.5,0.00000000001,theCall.Price,theCall.Vega)

0.4100000000000002

**Exercise 9.1** Modify the Newton-Raphson routine so that it does not endlessly loop if a root is not found.

For this exercise we will use the design pattern strategy, in order to externalize how part of one algorithm is implemented, using one of our terminator classes we developed previously in chapter 5.

**Exercise 9.2** Take your favourite numerical integration routine, e.g. the trapezium rule, and write a template routine to carry it out.

For this exercise we will code the trapezium rule.

In [202]:
def TrapezodialRule(Start, Finish, Intervals, Value):
    delta = (Finish - Start) / (Intervals)
    x = np.arange(Start, Finish + delta, delta)
    y = list(map(Value,x))
    total = y.pop(0)
    total += y.pop(-1)
    total += 2*sum(y)
    total *= delta/2
    return total

In [207]:
def quadratic(x):
    return 3*(x**2) - 2*x + 1

In [203]:
TrapezodialRule(0,5,100,quadratic)

105.00625000000002

We can also use a modification of the trapezodial rule without specifying how many intervals, just the Tolerance.

In [226]:
def TrapezodialRule(Start, Finish, Tolerance, Value):
    prev = -1
    Intervals = 2
    total = 0
    while abs(prev  - total) > Tolerance:
        prev = total
        delta = (Finish - Start) / (Intervals)
        x = np.arange(Start, Finish + delta, delta)
        y = list(map(Value,x))
        total = y.pop(0)
        total += y.pop(-1)
        total += 2*sum(y)
        total *= delta/2
        Intervals*=2
    return total

In [229]:
TrapezodialRule(0,5,0.000001,quadratic)

105.00000023283064

We could even use the strategy pattern to define why the routine stops, using a terminator object that can handle different conditions to stop such as the Tolerance or the runtime.

**Exercise 9.3** Write a routine to price a vanilla option by MonteCarlo or trees where the pay-off is passed in as a template parameter expressed via a function object.

Since there are no templates in python (because it is not a typed language) it does not make sense to do this exercise.